In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
data_train = np.load('augmented_data/pixelized.npz')
data_labels = np.load('augmented_data/pixelized_class.npz')

In [3]:
x_train = data_train['arr_0'][:4000]
x_labels = data_labels['arr_0'][:4000]


In [4]:
category_mapping = {90: 'SN1a', 67: 'SN1a-91bg', 52: 'SN1ax', 42: 'SN2', 62: 'SN1bc', 95: 'SLSN1', 15: 'TDE', 64: 'KN', 88: 'AGN', 92: 'RRL', 65: 'M-dwarf', 16: 'EB', 53: 'Mira', 6: 'Microlens'}

In [5]:
y_train = []
for i,obj_class in enumerate(x_labels):
    #class_ = meta_[meta_['object_id'] == obj_id]['target'].values[0]
    
    if category_mapping[obj_class] == 'AGN':
        y_train.append(1)
    else:
        y_train.append(0)
    
y_train = np.array(y_train)

In [6]:
# Model / data parameters
num_classes = 2
input_shape = (256, 1300, 6)

y_train = keras.utils.to_categorical(y_train, num_classes)

In [7]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 1298, 32)     1760      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 649, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 647, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 323, 64)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1281664)           0         
                                                                 
 dropout (Dropout)           (None, 1281664)           0

In [8]:
batch_size = 32
epochs = 50

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/50
113/113 [==============================] - 143s 1s/step - loss: 0.6457 - accuracy: 0.9481 - val_loss: 2.2406 - val_accuracy: 0.3650
Epoch 2/50
113/113 [==============================] - 144s 1s/step - loss: 0.5614 - accuracy: 0.9481 - val_loss: 2.1187 - val_accuracy: 0.3875
Epoch 3/50
113/113 [==============================] - 142s 1s/step - loss: 0.4929 - accuracy: 0.9481 - val_loss: 2.0094 - val_accuracy: 0.4075
Epoch 4/50
113/113 [==============================] - 142s 1s/step - loss: 0.4377 - accuracy: 0.9481 - val_loss: 1.9115 - val_accuracy: 0.4325
Epoch 5/50
113/113 [==============================] - 142s 1s/step - loss: 0.3932 - accuracy: 0.9481 - val_loss: 1.8238 - val_accuracy: 0.4550
Epoch 6/50
113/113 [==============================] - 142s 1s/step - loss: 0.3574 - accuracy: 0.9481 - val_loss: 1.7457 - val_accuracy: 0.4700
Epoch 7/50
113/113 [==============================] - 142s 1s/step - loss: 0.3285 - accuracy: 0.9481 - val_loss: 1.6747 - val_accuracy: 0.4875

KeyboardInterrupt: 